In [1]:
import julia 
from julia import Julia
jl = Julia(compiled_modules=False)  # Initialize Julia interpreter
Julia(runtime="/home/jxxiong/julia-1.9.2/bin/julia")
from julia import Base
# Now you can call functions from Julia's Base module
from julia import Main

In [2]:
import numpy as np
import gurobipy as gp

from gurobipy import Model, GRB, QuadExpr, LinExpr
from scipy.sparse import csc_matrix
import pickle
import os

In [3]:
case_dcit = {200: "pglib_opf_case200_activ.m",
             2000: "pglib_opf_case2000_goc.m",
             10000: "pglib_opf_case10000_goc.m",
             2312: "pglib_opf_case2312_goc.m",
             793: "pglib_opf_case793_goc.m",
             3: "pglib_opf_case3_lmbd.m",
             3970: "pglib_opf_case3970_goc.m"}

In [4]:
nbus = 3970
case_name = case_dcit[nbus]
Main.case_name = case_name
Main.nbus = nbus

In [5]:
Main.eval("""
using PowerModels
using JuMP
using Ipopt

file_path = "/home/jxxiong/A-xjx/DC3/datasets/dcopf/";
network_data_orig = PowerModels.parse_file(file_path*case_name);
optimizer = optimizer_with_attributes(Ipopt.Optimizer, "print_level"=>0);
""")

[info | PowerModels]: removing 1 cost terms from generator 306: [0.6494, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 54: [4237.0, -21.82]
[info | PowerModels]: removing 1 cost terms from generator 101: [2249.0, -37.75]
[info | PowerModels]: removing 1 cost terms from generator 371: [1.01, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 41: [1988.0, -1.055]
[info | PowerModels]: removing 1 cost terms from generator 65: [6202.0, -2.892]
[info | PowerModels]: removing 1 cost terms from generator 335: [2614.0, -125.8]
[info | PowerModels]: removing 1 cost terms from generator 362: [1948.9999999999998, -5.905]
[info | PowerModels]: removing 1 cost terms from generator 326: [4467.0, -18.59]
[info | PowerModels]: removing 1 cost terms from generator 299: [2815.0, -27.14]
[info | PowerModels]: removing 1 cost terms from generator 168: [2002.0, -72.15]
[info | PowerModels]: removing 1 cost terms from generator 159: [2055.0, -92.67]
[info | PowerModels]: remov

<PyCall.jlwrap MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])>

64.42]
[info | PowerModels]: removing 1 cost terms from generator 19: [4217.0, -40.34]
[info | PowerModels]: removing 1 cost terms from generator 179: [2023.0, -71.24]
[info | PowerModels]: removing 1 cost terms from generator 242: [9053.0, -13.71]
[info | PowerModels]: removing 1 cost terms from generator 260: [4004.9999999999995, -56.73]
[info | PowerModels]: removing 1 cost terms from generator 239: [0.2865, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 35: [0.2, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 317: [0.6494, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 197: [8988.0, -44.49]
[info | PowerModels]: removing 1 cost terms from generator 276: [0.641, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 316: [0.6494, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 365: [1.01, 0.0]
[info | PowerModels]: removing 1 cost terms from generator 21: [7029.000000000001, -1.145]
[info | PowerModels]: rem

In [6]:
def get_constraints(m):
    ini_ineq_matrix = []
    ini_ineq_rhs = []
    ini_eq_matrix = []
    ini_eq_rhs = []
    a_matrix = m.getA().toarray()

    i = 0
    for constr in m.getConstrs():
        if constr.sense == '<':
            ini_ineq_matrix.append(a_matrix[i,:])
            ini_ineq_rhs.append(constr.RHS)
            i += 1
        elif constr.sense == '>':
            ini_ineq_matrix.append(-a_matrix[i,:])
            ini_ineq_rhs.append(-constr.RHS)
            i += 1
        elif constr.sense == '=':
            ini_eq_matrix.append(a_matrix[i,:])
            ini_eq_rhs.append(constr.RHS)
            i += 1

    ini_ineq_matrix = np.array(ini_ineq_matrix)
    ini_ineq_rhs = np.array(ini_ineq_rhs)
    ini_eq_matrix = np.array(ini_eq_matrix)
    ini_eq_rhs = np.array(ini_eq_rhs)

    return ini_ineq_matrix, ini_ineq_rhs, ini_eq_matrix, ini_eq_rhs

def get_objective(m):
    ini_q_matrix = m.getQ().toarray()
    ini_p_vec = []
    variables = m.getVars()
    for var in variables:
        ini_p_vec.append(var.obj)
    ini_p_vec = np.array(ini_p_vec)

    return ini_q_matrix, ini_p_vec

def get_bound(m):
    variables = m.getVars()
    ini_lb = []
    ini_ub = []
    for var in variables:
        ini_lb.append(var.lb)
        ini_ub.append(var.ub)
    ini_lb = np.array(ini_lb)
    ini_ub = np.array(ini_ub)

    return ini_lb, ini_ub

## Get Q, p, A, G, h

In [7]:
Main.eval("""
pm = instantiate_model(network_data_orig, DCPPowerModel, PowerModels.build_opf);
JuMP.write_to_file(pm.model, "dcopf$(nbus)_model.mps")
""")

In [8]:
m = gp.read(f'/home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf{nbus}_model.mps')
G, h, A, _ = get_constraints(m)
Q, p = get_objective(m)
Lb, Ub = get_bound(m)

Read MPS format model from file /home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf3970_model.mps
Reading time = 0.02 seconds
: 22036 rows, 10994 columns, 56437 nonzeros


In [9]:
print(G.shape, h.shape, A.shape, Q.shape, p.shape, Lb.shape, Ub.shape)

(11424, 10994) (11424,) (10612, 10994) (10994, 10994) (10994,) (10994,) (10994,)


In [10]:
np.where(Lb == Ub)[0]

array([], dtype=int64)

In [11]:
m.getQ().toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
Main.eval("""result = optimize_model!(pm, optimizer=optimizer);""")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************



{'solve_time': 3.224604845046997,
 'optimizer': 'Ipopt',
 'termination_status': <PyCall.jlwrap LOCALLY_SOLVED>,
 'dual_status': <PyCall.jlwrap FEASIBLE_POINT>,
 'primal_status': <PyCall.jlwrap FEASIBLE_POINT>,
 'objective': 934219.288870433,
 'solution': {'baseMVA': 100.0,
  'branch': {'4304': {'qf': nan,
    'qt': nan,
    'pt': -0.12057640202297033,
    'pf': 0.12057640202297033},
   '5422': {'qf': nan,
    'qt': nan,
    'pt': 0.05128753858267124,
    'pf': -0.05128753858267124},
   '5734': {'qf': nan,
    'qt': nan,
    'pt': 0.07744501018206096,
    'pf': -0.07744501018206096},
   '3935': {'qf': nan,
    'qt': nan,
    'pt': 0.3450018261384974,
    'pf': -0.3450018261384974},
   '2243': {'qf': nan,
    'qt': nan,
    'pt': -0.1359517378346961,
    'pf': 0.1359517378346961},
   '1881': {'qf': nan,
    'qt': nan,
    'pt': -0.3075752967965761,
    'pf': 0.3075752967965761},
   '5425': {'qf': nan,
    'qt': nan,
    'pt': -0.0346486964321994,
    'pf': 0.0346486964321994},
   '4209':

In [13]:
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 22036 rows, 10994 columns and 56437 nonzeros
Model fingerprint: 0x2092f4ae
Model has 65 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e-02, 1e+04]
  QObjective range [2e+02, 2e+04]
  Bounds range     [1e-03, 2e+01]
  RHS range        [1e-04, 9e-01]
Presolve removed 14759 rows and 5890 columns
Presolve time: 0.05s
Presolved: 7277 rows, 7551 columns, 32419 nonzeros
Presolved model has 65 quadratic objective terms
Ordering time: 0.01s

Barrier statistics:
 Free vars  : 313
 AA' NZ     : 8.316e+04
 Factor NZ  : 4.540e+05 (roughly 10 MB of memory)
 Factor Ops : 4.575e+07 (less than 1 second per iteration)
 Threads    : 24

                  Objective                Residu

## Generate eq_rhs (X)

In [14]:
num_samples = 1
perturb = 0.1
X = []
Y = []

Main.perturb = perturb

if not os.path.exists(f"dcopf{nbus}"):
    os.makedirs(f"dcopf{nbus}")

In [18]:
while len(X) < num_samples:
    Main.eval("""
        network_data = deepcopy(network_data_orig)
        for (load_id, load) in network_data["load"]
            perturbation_factor = (1 - perturb) + (rand() * perturb * 2);
            load["pd"] *= perturbation_factor;
            load["qd"] *= perturbation_factor;
        end

        pm = instantiate_model(network_data, DCPPowerModel, PowerModels.build_opf);
        result = optimize_model!(pm, optimizer=optimizer);
        status = (result["termination_status"] == MOI.LOCALLY_SOLVED || result["termination_status"] == MOI.OPTIMAL);
    """)

    if Main.status:
        print(Main.status)
        Main.eval("""JuMP.write_to_file(pm.model, "dcopf$(nbus)_model.mps")""")
        m = gp.read(f'/home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf{nbus}_model.mps')
        m.optimize()
        m.setParam('OutputFlag', 0)
        if m.status == GRB.Status.OPTIMAL:
            _, _, _, x = get_constraints(m)
            X.append(x)
            Y.append([var.x for var in m.getVars()])
    
    if len(X) % 100 == 0 and len(X) != 0:
        # save the data A, X, G, h, Q, p, lb, ub
        data = {'Q':Q, 'p':p, 'A':A, 'X':X, 'G':G, 'h':h, 'Lb':Lb, 'Ub':Ub, 'Y':[]}
        with open(f"dcopf{nbus}_data", 'wb') as f:
            pickle.dump(data, f)
    if len(X) % 1000 == 0 and len(X) != 0:
        data = {'Q':Q, 'p':p, 'A':A, 'X':X, 'G':G, 'h':h, 'Lb':Lb, 'Ub':Ub, 'Y':Y}
        with open(f"dcopf{nbus}/dcopf{nbus}_data_{len(X)}", 'wb') as f:
            pickle.dump(data, f)

with open(f"dcopf{nbus}_data", 'wb') as f:
    pickle.dump(data, f)


True
Read MPS format model from file /home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf3970_model.mps
Reading time = 0.02 seconds
: 22036 rows, 10994 columns, 56437 nonzeros
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (linux64 - "Ubuntu 22.04.2 LTS")

CPU model: 12th Gen Intel(R) Core(TM) i9-12900K, instruction set [SSE2|AVX|AVX2]
Thread count: 24 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 22036 rows, 10994 columns and 56437 nonzeros
Model fingerprint: 0x753aeda4
Model has 65 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [1e-02, 1e+04]
  QObjective range [2e+02, 2e+04]
  Bounds range     [1e-03, 2e+01]
  RHS range        [9e-05, 9e-01]


Presolve removed 14762 rows and 5892 columns
Presolve time: 0.05s
Presolved: 7274 rows, 7548 columns, 32406 nonzeros
Presolved model has 65 quadratic objective terms
Ordering time: 0.01s

Barrier statistics:
 Free vars  : 274
 AA' NZ     : 8.315e+04
 Factor NZ  : 4.442e+05 (roughly 10 MB of memory)
 Factor Ops : 4.356e+07 (less than 1 second per iteration)
 Threads    : 24

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   6.83461960e+09 -2.78755596e+10  8.50e+03 1.00e+03  1.03e+06     0s
   1   1.77074288e+09 -1.68075091e+10  3.80e+03 4.47e+02  4.69e+05     0s
   2   5.73068649e+07 -1.57904974e+10  3.25e+02 3.83e+01  5.85e+04     0s
   3   5.51730745e+06 -6.54000876e+09  2.99e+01 3.52e+00  6.61e+03     0s
   4   1.70333655e+06 -3.33421178e+09  4.57e+00 5.37e-01  1.51e+03     0s
   5   1.37536659e+06 -1.95683168e+09  1.99e+00 2.34e-01  7.22e+02     0s
   6   1.26922825e+06 -1.27321209e+09  4.70e-01 5.53e

NameError: name 'data' is not defined

In [16]:
Main.result["objective"]

931855.8945191033

## Load Data

In [ ]:
with open(f"dcopf{nbus}_data", 'rb') as f:
    dataset = pickle.load(f)

# Other

In [10]:
m = gp.read(f'/home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf{nbus}_model.mps')

Read LP format model from file /home/jxxiong/A-xjx/DC3/datasets/dcopf/dcopf793_model.lp
Reading time = 0.01 seconds
obj: 3515 rows, 1804 columns, 8279 nonzeros


In [15]:
G, h, A, x = get_constraints(m)
Q, p = get_objective(m)
Lb, Ub = get_bound(m)

In [16]:
np.where(G !=0 )

(array([   0,    0,    1, ..., 1806, 1807, 1807]),
 array([ 98,  99, 100, ..., 612, 155, 335]))

In [17]:
np.where(Q != 0)

(array([ 1,  4,  5,  6,  7, 10, 13, 15, 17, 18, 19, 21, 24, 29, 30, 31, 32,
        33, 39, 40, 41, 42, 49, 50, 53, 54, 55, 59, 60, 61, 67, 69, 70, 71,
        72, 77, 78, 80, 81, 84, 85, 86, 89, 91, 92, 93, 96, 97]),
 array([ 1,  4,  5,  6,  7, 10, 13, 15, 17, 18, 19, 21, 24, 29, 30, 31, 32,
        33, 39, 40, 41, 42, 49, 50, 53, 54, 55, 59, 60, 61, 67, 69, 70, 71,
        72, 77, 78, 80, 81, 84, 85, 86, 89, 91, 92, 93, 96, 97]))

In [18]:
np.where(p!=0)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95]),)

In [ ]:
nonzero_indices = np.nonzero(Q)
print(nonzero_indices)


In [ ]:
np.nonzero(p)

In [ ]:
lb_index = np.where(Lb != -np.inf)[0]
ub_index = np.where(Ub != np.inf)[0]

In [ ]:
np.where(Lb == Ub)[0]

In [ ]:
intersection = np.intersect1d(lb_index, ub_index)
print(intersection)

In [ ]:
np.mean(data['objective'])

In [ ]:
np.mean(data['solve_time'])

In [ ]:
len(data['objective'])